In [3]:
import pandas as pd
import spacy
from collections import Counter

# Load spaCy model for NER
nlp = spacy.load("en_core_web_lg")

In [10]:
import spacy
from spacy.matcher import PhraseMatcher

def extract_locations(text):
    nlp = spacy.load("en_core_web_sm")
    matcher = PhraseMatcher(nlp.vocab)
    # Add custom location names to the matcher
    location_patterns = ['Gandhinagar']
    for loc in location_patterns:
        matcher.add("LOCATION", None, nlp(loc))

    doc = nlp(text)
    locations = []
    
    # Extract locations using NER
    for ent in doc.ents:
        if ent.label_ == 'GPE' or ent.label_ == 'LOC':
            locations.append(ent.text)

    # Extract additional locations using the custom matcher
    matches = matcher(doc)
    for match_id, start, end in matches:
        span = doc[start:end]
        locations.append(span.text)
    
    return locations

paragraph = """
UN statue of liberty reports iraq tokyo Leogane Gandhinagar 80-90 destroyed.
"""

locations = extract_locations(paragraph)
print("Locations mentioned in the paragraph:")
for location in locations:
    print(location)


Locations mentioned in the paragraph:
iraq
tokyo
Gandhinagar


In [1]:
!pip install tqdm

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from tqdm import tqdm

In [27]:
import pandas as pd
import spacy
from spacy.matcher import PhraseMatcher

def extract_locations(text):
    nlp = spacy.load("en_core_web_sm")
    matcher = PhraseMatcher(nlp.vocab)
    # Add custom location names to the matcher
    location_patterns = ['Gandhinagar']
    for loc in location_patterns:
        matcher.add("LOCATION", None, nlp(loc))

    doc = nlp(text)
    locations = []
    
    # Extract locations using NER
    for ent in doc.ents:
        if ent.label_ == 'GPE' or ent.label_ == 'LOC':
            locations.append(ent.text)
    # Extract additional locations using the custom matcher
    matches = matcher(doc)
    for match_id, start, end in matches:
        span = doc[start:end]
        locations.append(span.text)
        
    return locations

In [ ]:
# Example DataFrame
# data = {'processed_text': ["UN statue of liberty reports iraq tokyo Leogane Gandhinagar 80-90 destroyed."]}
df = pd.read_csv('preprocessed_0f_iraq_iran.csv')

# Function to extract locations from each entry in the processed_text column
def extract_locations_from_text(text):
    locations = extract_locations(text)
    return locations

# Apply extract_locations_from_dataframe to each row and flatten the result
tqdm.pandas()
locations_array = df['processed_text'].progress_apply(extract_locations_from_text).explode()


# Count the frequency of each location
location_counts = locations_array.value_counts()

print("Locations mentioned in the DataFrame along with their frequencies:")
print(location_counts)


In [23]:
locations_array

0      iran
1      iraq
1      iran
2      iran
3      iraq
       ... 
592    iran
593    iran
594    iran
595    iran
596     NaN
Name: processed_text, Length: 891, dtype: object

In [14]:
import streamlit as st
import pandas as pd
import folium
from streamlit_folium import folium_static
from geopy.geocoders import Nominatim
import spacy
from spacy.matcher import PhraseMatcher
from collections import Counter
import itertools

In [30]:
def plot_map_with_frequencies(locations_array, save_as_html):
    # Count the frequency of each location
    location_counter = Counter(locations_array)
    max_frequency = max(location_counter.values())
    
    # Create a folium map centered at the mean coordinates of all locations
    map_center = [locations_array.mean(axis=0)]
    m = folium.Map(location=map_center, zoom_start=4)

    # Add markers for each location with color based on frequency
    for location, frequency in location_counter.items():
        coordinates = get_coordinates(location)
        if coordinates:
            color = get_color(frequency, max_frequency)
            # Repeat the location name based on its frequency
            for _ in range(frequency):
                folium.CircleMarker(location=coordinates, radius=3, color=color, fill=True, fill_opacity=0.7).add_to(m)

    # Save the map as HTML with the specified name
    m.save(save_as_html)
    print(f"Map saved as '{save_as_html}'")


In [16]:
def get_coordinates(location_name):
    geolocator = Nominatim(user_agent="location_converter")
    location = geolocator.geocode(location_name)
    if location:
        return location.latitude, location.longitude
    else:
        return None

def get_color(frequency, max_frequency):
    if frequency == 1:
        return 'green'
    elif frequency == 2:
        return 'blue'
    else:
        return 'red'

In [28]:
data = {'processed_text': ["UN statue of liberty reports iraq tokyo Leogane Gandhinagar 80-90 destroyed by tokyo."]}
df1 = pd.DataFrame(data)

In [29]:
tqdm.pandas()
locations_array1 = df1['processed_text'].progress_apply(extract_locations_from_text).explode()


# Count the frequency of each location
location_counts1 = locations_array1.value_counts()

print("Locations mentioned in the DataFrame along with their frequencies:")
print(location_counts1)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.04it/s]

Locations mentioned in the DataFrame along with their frequencies:
processed_text
tokyo          2
iraq           1
Gandhinagar    1
Name: count, dtype: int64


In [31]:
save_as_html = "map.html"

# Generate and save the map
plot_map_with_frequencies(locations_array1, save_as_html)

TypeError: Could not convert string 'iraqtokyotokyoGandhinagar' to numeric